In [1]:
import argparse
import os
import json
import sys
from codebase.dataset import Dataset
from codebase.models import *
from codebase.trainer import Trainer, make_dir_if_not_exist
from codebase.defaults import get_default_kwargs
import tensorflow as tf


2024-01-15 17:00:09.935846: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 17:00:09.992685: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 17:00:09.992747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 17:00:09.994364: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 17:00:10.006086: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 17:00:10.007298: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
parser = argparse.ArgumentParser(description='Run model')
parser.add_argument('-d', '--dataset', help='dataset name', default='zindi_sex')
parser.add_argument('-n', '--expname', help='experiment name', default='temp')
parser.add_argument('-pc', '--pass_coeff', help='coefficient on pass term', default=1., type=float)
parser.add_argument('-fc', '--fair_coeff', help='coefficient on fairness term', default=1., type=float)
parser.add_argument('-pass', '--pass_model', action='store_true', help='run PASS model?', default=False)
parser.add_argument('-def', '--defer', action='store_true', help='learn to defer? (as opposed to reject)', default=True)
parser.add_argument('-ne', '--num_epochs', help='number of training epochs', default=10000, type=int)
parser.add_argument('-pa', '--patience', help='training patience for early stopping', default=30, type=int)
parser.add_argument('-ua', '--use_attr', action='store_true', help='use sensitive attribute in data', default=True)
parser.add_argument('-mrs', '--model_random_seed', help='what random seed to use for model', default=0, type=int)
parser.add_argument('-drs', '--data_random_seed', help='what random seed to use for data', default=0, type=int)
parser.add_argument('-dm', '--dm_type', help='type of DM to use: inconsistent/biased/highacc', default='inconsistent')
parser.add_argument('-dirs', '--dirconf', help='config file for dirs', default='conf/dirs/home.json')
args = vars(parser.parse_args([]))


In [3]:
#load options
dirs = json.load(open(args['dirconf'], 'r'))
model_args = json.load(open('conf/model/{}.json'.format(args['dataset'])))

# get default params
data_kwargs, model_kwargs = get_default_kwargs(dirs['data'], args, model_args)

In [6]:
#change data kwargs
experiment_info = {}
experiment_info["K"] = []
experiment_info["budget"] = []
experiment_info["Error"] = []
experiment_info["Equal opportunity difference"] = []
experiment_info["fc"] = []

fcs = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1,1.5,2,3,4,5]
for fc in fcs:
    model_kwargs["fair_coeff"] = fc
    for K in [1,2,3,4,5]:
        for budget in [1, 0.1, 0.01]:
            tf.compat.v1.reset_default_graph()
            data_kwargs['npzfile'] = "data/zindi/zindi{}_sex.npz".format(K,budget)
            #get dataset
            data = Dataset(**data_kwargs,budget=budget)
            model = MLPDeferModel(**model_kwargs)

            with tf.compat.v1.Session() as sess:
                print('Session created.')
                resdirname = os.path.join(dirs['exp'], args['dataset'])
                logdirname = os.path.join(dirs['log'], args['dataset'])
                ckptdirname = os.path.join(resdirname, 'checkpoints')
                for d in [resdirname, logdirname, ckptdirname]:
                    make_dir_if_not_exist(d)
                #create Trainer
                trainer = Trainer(model, data, sess=sess, logs_path=logdirname, \
                            checkpoint_path=ckptdirname, results_path=resdirname)

                save_path = trainer.train(n_epochs=args['num_epochs'], patience=args['patience'])
                trainer.restore(save_path)
                error,eod = trainer.test()
                print('Complete.')
                experiment_info["K"].append(K)
                experiment_info["budget"].append(budget)
                experiment_info["Error"].append(error)
                experiment_info["Equal opportunity difference"].append(eod)
                experiment_info["fc"].append(fc)


Session created.
Train: E0: loss:0.430, class_loss:0.323, fair_reg:0.134, idk_loss:0.107, loss_class:0.323, loss_idk:0.068, errRate:0.130, DI:0.052, errRate-T:0.128, DI-T:0.059, rejErrRate:0.129, IDKRate:0.069 | Valid: E0: loss:0.298, class_loss:0.290, fair_reg:0.185, idk_loss:0.008, loss_class:0.290, loss_idk:0.006, errRate:0.102, DI:0.030, errRate-T:0.102, DI-T:0.030, rejErrRate:0.102, IDKRate:0.000
Best validation loss so far of 0.298, model saved to /home/hc838/AuFair/predict-responsibly/experiments/zindi_sex/checkpoints/model.ckpt
Train: E1: loss:0.301, class_loss:0.297, fair_reg:0.168, idk_loss:0.004, loss_class:0.297, loss_idk:0.003, errRate:0.120, DI:0.066, errRate-T:0.120, DI-T:0.066, rejErrRate:0.120, IDKRate:0.000 | Valid: E1: loss:0.291, class_loss:0.289, fair_reg:0.195, idk_loss:0.002, loss_class:0.289, loss_idk:0.002, errRate:0.102, DI:0.043, errRate-T:0.102, DI-T:0.043, rejErrRate:0.102, IDKRate:0.000
Train: E2: loss:0.297, class_loss:0.295, fair_reg:0.170, idk_loss:0.00

Train: E20: loss:0.276, class_loss:0.276, fair_reg:0.187, idk_loss:0.000, loss_class:0.276, loss_idk:0.000, errRate:0.112, DI:0.066, errRate-T:0.112, DI-T:0.066, rejErrRate:0.112, IDKRate:0.000 | Valid: E20: loss:0.285, class_loss:0.285, fair_reg:0.196, idk_loss:0.000, loss_class:0.285, loss_idk:0.000, errRate:0.104, DI:0.010, errRate-T:0.104, DI-T:0.010, rejErrRate:0.104, IDKRate:0.000
Train: E21: loss:0.275, class_loss:0.275, fair_reg:0.188, idk_loss:0.000, loss_class:0.275, loss_idk:0.000, errRate:0.112, DI:0.066, errRate-T:0.112, DI-T:0.066, rejErrRate:0.112, IDKRate:0.000 | Valid: E21: loss:0.286, class_loss:0.286, fair_reg:0.196, idk_loss:0.000, loss_class:0.286, loss_idk:0.000, errRate:0.103, DI:0.008, errRate-T:0.103, DI-T:0.008, rejErrRate:0.103, IDKRate:0.000
Train: E22: loss:0.275, class_loss:0.275, fair_reg:0.188, idk_loss:0.000, loss_class:0.275, loss_idk:0.000, errRate:0.112, DI:0.067, errRate-T:0.112, DI-T:0.067, rejErrRate:0.112, IDKRate:0.000 | Valid: E22: loss:0.286, 

Train: E42: loss:0.269, class_loss:0.269, fair_reg:0.194, idk_loss:0.000, loss_class:0.269, loss_idk:0.000, errRate:0.110, DI:0.062, errRate-T:0.110, DI-T:0.062, rejErrRate:0.110, IDKRate:0.000 | Valid: E42: loss:0.290, class_loss:0.290, fair_reg:0.196, idk_loss:0.000, loss_class:0.290, loss_idk:0.000, errRate:0.104, DI:0.022, errRate-T:0.104, DI-T:0.022, rejErrRate:0.104, IDKRate:0.000
Finished training: min validation loss was 0.284 in epoch 12
INFO:tensorflow:Restoring parameters from /home/hc838/AuFair/predict-responsibly/experiments/zindi_sex/checkpoints/model.ckpt
0.2671480144307889
0.33080808079972707
Error rate is 0.11777210884353742
Equal opportunity difference is 0.06366006636893817
Test: Test: loss:0.293, class_loss:0.293, fair_reg:0.157, idk_loss:0.000, loss_class:0.293, loss_idk:0.000, errRate:0.118, DI:0.036, errRate-T:0.118, DI-T:0.036, rejErrRate:0.118, IDKRate:0.000
Metrics saved to /home/hc838/AuFair/predict-responsibly/experiments/zindi_sex/test_results.csv
Tensors s

Train: E18: loss:0.263, class_loss:0.261, fair_reg:0.185, idk_loss:0.002, loss_class:0.261, loss_idk:0.002, errRate:0.100, DI:0.051, errRate-T:0.100, DI-T:0.051, rejErrRate:0.100, IDKRate:0.000 | Valid: E18: loss:0.207, class_loss:0.204, fair_reg:0.283, idk_loss:0.002, loss_class:0.204, loss_idk:0.002, errRate:0.063, DI:0.052, errRate-T:0.063, DI-T:0.052, rejErrRate:0.063, IDKRate:0.000
Train: E19: loss:0.262, class_loss:0.260, fair_reg:0.186, idk_loss:0.002, loss_class:0.260, loss_idk:0.002, errRate:0.100, DI:0.045, errRate-T:0.100, DI-T:0.045, rejErrRate:0.100, IDKRate:0.000 | Valid: E19: loss:0.207, class_loss:0.205, fair_reg:0.291, idk_loss:0.002, loss_class:0.205, loss_idk:0.002, errRate:0.063, DI:0.052, errRate-T:0.063, DI-T:0.052, rejErrRate:0.063, IDKRate:0.000
Train: E20: loss:0.261, class_loss:0.259, fair_reg:0.187, idk_loss:0.002, loss_class:0.259, loss_idk:0.002, errRate:0.100, DI:0.045, errRate-T:0.100, DI-T:0.045, rejErrRate:0.100, IDKRate:0.000 | Valid: E20: loss:0.207, 

Train: E40: loss:0.242, class_loss:0.241, fair_reg:0.208, idk_loss:0.001, loss_class:0.241, loss_idk:0.000, errRate:0.094, DI:0.045, errRate-T:0.094, DI-T:0.045, rejErrRate:0.094, IDKRate:0.000 | Valid: E40: loss:0.211, class_loss:0.210, fair_reg:0.285, idk_loss:0.001, loss_class:0.210, loss_idk:0.000, errRate:0.070, DI:0.022, errRate-T:0.070, DI-T:0.022, rejErrRate:0.070, IDKRate:0.000
Train: E41: loss:0.240, class_loss:0.240, fair_reg:0.208, idk_loss:0.001, loss_class:0.240, loss_idk:0.000, errRate:0.093, DI:0.046, errRate-T:0.093, DI-T:0.046, rejErrRate:0.093, IDKRate:0.000 | Valid: E41: loss:0.211, class_loss:0.210, fair_reg:0.284, idk_loss:0.001, loss_class:0.210, loss_idk:0.000, errRate:0.070, DI:0.022, errRate-T:0.070, DI-T:0.022, rejErrRate:0.070, IDKRate:0.000
Train: E42: loss:0.239, class_loss:0.239, fair_reg:0.208, idk_loss:0.001, loss_class:0.239, loss_idk:0.000, errRate:0.092, DI:0.040, errRate-T:0.092, DI-T:0.040, rejErrRate:0.092, IDKRate:0.000 | Valid: E42: loss:0.211, 

...

Train: E35: loss:0.269, class_loss:0.248, fair_reg:0.020, idk_loss:0.001, loss_class:0.248, loss_idk:0.001, errRate:0.105, DI:0.030, errRate-T:0.105, DI-T:0.030, rejErrRate:0.105, IDKRate:0.000 | Valid: E35: loss:0.411, class_loss:0.244, fair_reg:0.166, idk_loss:0.001, loss_class:0.244, loss_idk:0.001, errRate:0.096, DI:0.193, errRate-T:0.096, DI-T:0.193, rejErrRate:0.096, IDKRate:0.000
Train: E36: loss:0.275, class_loss:0.244, fair_reg:0.031, idk_loss:0.001, loss_class:0.244, loss_idk:0.001, errRate:0.101, DI:0.018, errRate-T:0.101, DI-T:0.018, rejErrRate:0.101, IDKRate:0.000 | Valid: E36: loss:0.416, class_loss:0.246, fair_reg:0.170, idk_loss:0.001, loss_class:0.246, loss_idk:0.001, errRate:0.096, DI:0.193, errRate-T:0.096, DI-T:0.193, rejErrRate:0.096, IDKRate:0.000
Finished training: min validation loss was 0.373 in epoch 6
INFO:tensorflow:Restoring parameters from /home/hc838/AuFair/predict-responsibly/experiments/zindi_sex/checkpoints/model.ckpt
0.0
0.0
Error rate is 0.1460148777

Train: E17: loss:0.547, class_loss:0.330, fair_reg:0.046, idk_loss:0.171, loss_class:0.330, loss_idk:0.116, errRate:0.125, DI:0.000, errRate-T:0.125, DI-T:0.000, rejErrRate:0.125, IDKRate:0.000 | Valid: E17: loss:0.491, class_loss:0.324, fair_reg:0.009, idk_loss:0.157, loss_class:0.324, loss_idk:0.107, errRate:0.100, DI:0.000, errRate-T:0.100, DI-T:0.000, rejErrRate:0.100, IDKRate:0.000
Train: E18: loss:0.576, class_loss:0.326, fair_reg:0.095, idk_loss:0.155, loss_class:0.326, loss_idk:0.105, errRate:0.125, DI:0.000, errRate-T:0.125, DI-T:0.000, rejErrRate:0.125, IDKRate:0.000 | Valid: E18: loss:0.483, class_loss:0.325, fair_reg:0.016, idk_loss:0.142, loss_class:0.325, loss_idk:0.096, errRate:0.100, DI:0.000, errRate-T:0.100, DI-T:0.000, rejErrRate:0.100, IDKRate:0.000
Best validation loss so far of 0.483, model saved to /home/hc838/AuFair/predict-responsibly/experiments/zindi_sex/checkpoints/model.ckpt
Train: E19: loss:0.496, class_loss:0.323, fair_reg:0.034, idk_loss:0.140, loss_clas

Train: E37: loss:0.312, class_loss:0.262, fair_reg:0.012, idk_loss:0.039, loss_class:0.262, loss_idk:0.029, errRate:0.125, DI:0.000, errRate-T:0.125, DI-T:0.000, rejErrRate:0.125, IDKRate:0.000 | Valid: E37: loss:0.405, class_loss:0.317, fair_reg:0.051, idk_loss:0.037, loss_class:0.317, loss_idk:0.025, errRate:0.100, DI:0.000, errRate-T:0.100, DI-T:0.000, rejErrRate:0.100, IDKRate:0.000
Best validation loss so far of 0.405, model saved to /home/hc838/AuFair/predict-responsibly/experiments/zindi_sex/checkpoints/model.ckpt
Train: E38: loss:0.372, class_loss:0.259, fair_reg:0.076, idk_loss:0.037, loss_class:0.259, loss_idk:0.027, errRate:0.125, DI:0.000, errRate-T:0.125, DI-T:0.000, rejErrRate:0.125, IDKRate:0.000 | Valid: E38: loss:0.407, class_loss:0.314, fair_reg:0.057, idk_loss:0.035, loss_class:0.314, loss_idk:0.024, errRate:0.100, DI:0.000, errRate-T:0.100, DI-T:0.000, rejErrRate:0.100, IDKRate:0.000
Train: E39: loss:0.299, class_loss:0.256, fair_reg:0.008, idk_loss:0.035, loss_clas

Train: E57: loss:0.301, class_loss:0.219, fair_reg:0.062, idk_loss:0.020, loss_class:0.219, loss_idk:0.015, errRate:0.117, DI:0.064, errRate-T:0.117, DI-T:0.064, rejErrRate:0.117, IDKRate:0.000 | Valid: E57: loss:0.402, class_loss:0.307, fair_reg:0.075, idk_loss:0.019, loss_class:0.307, loss_idk:0.013, errRate:0.100, DI:0.000, errRate-T:0.100, DI-T:0.000, rejErrRate:0.100, IDKRate:0.000
Train: E58: loss:0.250, class_loss:0.218, fair_reg:0.013, idk_loss:0.019, loss_class:0.218, loss_idk:0.015, errRate:0.117, DI:0.064, errRate-T:0.117, DI-T:0.064, rejErrRate:0.117, IDKRate:0.000 | Valid: E58: loss:0.400, class_loss:0.309, fair_reg:0.072, idk_loss:0.019, loss_class:0.309, loss_idk:0.013, errRate:0.100, DI:0.000, errRate-T:0.100, DI-T:0.000, rejErrRate:0.100, IDKRate:0.000
Train: E59: loss:0.252, class_loss:0.216, fair_reg:0.018, idk_loss:0.018, loss_class:0.216, loss_idk:0.015, errRate:0.117, DI:0.064, errRate-T:0.117, DI-T:0.064, rejErrRate:0.117, IDKRate:0.000 | Valid: E59: loss:0.400, 

Train: E79: loss:0.231, class_loss:0.192, fair_reg:0.028, idk_loss:0.011, loss_class:0.192, loss_idk:0.009, errRate:0.075, DI:0.064, errRate-T:0.075, DI-T:0.064, rejErrRate:0.075, IDKRate:0.000 | Valid: E79: loss:0.422, class_loss:0.310, fair_reg:0.101, idk_loss:0.011, loss_class:0.310, loss_idk:0.008, errRate:0.100, DI:0.000, errRate-T:0.100, DI-T:0.000, rejErrRate:0.100, IDKRate:0.000
Train: E80: loss:0.226, class_loss:0.190, fair_reg:0.025, idk_loss:0.011, loss_class:0.190, loss_idk:0.009, errRate:0.075, DI:0.064, errRate-T:0.075, DI-T:0.064, rejErrRate:0.075, IDKRate:0.000 | Valid: E80: loss:0.423, class_loss:0.311, fair_reg:0.101, idk_loss:0.011, loss_class:0.311, loss_idk:0.007, errRate:0.100, DI:0.000, errRate-T:0.100, DI-T:0.000, rejErrRate:0.100, IDKRate:0.000
Finished training: min validation loss was 0.380 in epoch 50
INFO:tensorflow:Restoring parameters from /home/hc838/AuFair/predict-responsibly/experiments/zindi_sex/checkpoints/model.ckpt
0.006211180124030709
0.0602739726

In [8]:
import pandas as pd
df = pd.DataFrame.from_dict(experiment_info)
#save df to csv
df.to_csv("results/zindi_sex.csv",index=False)